In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import warnings
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

In [12]:
df=pd.read_csv("dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df['isFraud'].value_counts()

In [ ]:
df['isFlaggedFraud'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape[0]

In [7]:
round((df['isFraud'].value_counts()[1] / df.shape[0]) * 100,2)

NameError: name 'df' is not defined

# Transaction Types with their count

In [ ]:
df["type"].value_counts().plot(kind="bar",title="Transaction Types",color="orange")
plt.xlabel("Transaction Type")
plt.ylabel("Count")
plt.show()

# Fraud rates by types

In [ ]:
fraud_by_type=df.groupby("type")["isFraud"].mean().sort_values(ascending=False)
fraud_by_type.plot(kind="bar", title="Fraud Rate by Type", color="blue")
plt.ylabel("Fraud Rate")
plt.show()

In [ ]:
df["amount"].describe().astype(int)

In [ ]:
sns.histplot(np.log1p(df["amount"]),bins=100,kde=True,color="green")
plt.title("Transaction Amount Distribution (log scale)")
plt.xlabel("Log(Amount + 1)")
plt.show()

In [ ]:
sns.boxplot(data=df[df["amount"] < 50000], x="isFraud",y="amount")
plt.title("Amount vs isFraud (Filtered under 50k)")
plt.show()

In [ ]:
df.columns

In [ ]:
df["balanceDiffOrig"]= df["oldbalanceOrg"] - df["newbalanceOrig"]
df["balanceDiffDest"]= df["newbalanceDest"] - df["oldbalanceDest"]

In [ ]:
(df["balanceDiffOrig"] < 0).sum()

In [ ]:
(df["balanceDiffDest"] < 0).sum()

# Frauds over Time

In [ ]:
frauds_per_step=df[df["isFraud"] == 1]["step"].value_counts().sort_index()
plt.plot(frauds_per_step.index , frauds_per_step.values, label="Frauds per step")
plt.xlabel("Step (Time)")
plt.ylabel("Number of Frauds")
plt.title("Frauds over Time")
plt.show()

In [8]:
df.drop(columns="step",inplace=True)

NameError: name 'df' is not defined

In [ ]:
df.head()

In [ ]:
top_senders=df["nameOrig"].value_counts().head(10)

In [ ]:
top_senders

In [ ]:
top_receivers=df["nameDest"].value_counts().head(10)

In [ ]:
top_receivers

In [ ]:
fraud_users=df[df["isFraud"]==1]["nameOrig"].value_counts().head(10)

In [ ]:
fraud_users

In [ ]:
fraud_types=df[df["type"].isin(["TRANSFER","CASH_OUT"])]

In [ ]:
fraud_types["type"].value_counts()

In [ ]:
sns.countplot(data=fraud_types,x="type",hue="isFraud")
plt.title("Fraud Distribution in Transfer & Cash Out")
plt.show()

In [ ]:
corr = df[["amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest", "isFraud"]].corr()

In [ ]:
corr

In [ ]:
sns.heatmap(corr,annot=True,cmap="coolwarm",fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split #splitting
from sklearn.preprocessing import StandardScaler #scaling
from sklearn.linear_model import LogisticRegression #modeling
from sklearn.metrics import classification_report,confusion_matrix #evaluation
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer   #transformation
from sklearn.preprocessing import OneHotEncoder #transformation

In [9]:
df_model=df.drop(["nameOrig","nameDest","isFlaggedFraud"],axis=1)

NameError: name 'df' is not defined

In [ ]:
df_model.head()

In [ ]:
categorical=["type"]
numeric=["amount","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest"]

In [ ]:
y=df_model["isFraud"]
X=df_model.drop("isFraud",axis=1)

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,stratify=y)    #30% testing set

In [ ]:
preprocessor= ColumnTransformer(
    transformers=[
        ("num",StandardScaler(),numeric),
        ("cat", OneHotEncoder(drop="first"),categorical)
    ],
    remainder="drop"
)

In [ ]:
pipeline=Pipeline([
    ("prep", preprocessor),
    ("clf", LogisticRegression(class_weight="balanced", max_iter=1000))
])

In [ ]:
pipeline.fit(X_train,y_train)

In [ ]:
y_pred=pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
pipeline.score(X_test,y_test) * 100

In [ ]:
import joblib
joblib.dump(pipeline,"fraud_detection_pipeline.pkl")